In [ ]:
import math
import time
import threading
import numpy as np

from autofunc import AutoFunc
from python_api_class import PythonAPI
from concurrent.futures import ThreadPoolExecutor

from enum import Enum 

class MoveDirection(Enum): 
    Diagonal =      0
    Vertical =      1
    Horizontal =    2

In [ ]:
api_handle = PythonAPI()
af = AutoFunc(api_handle)

In [ ]:
# api_handle.send_command(f"sim")
api_handle.send_command("no_detect add {} {} {} {}".format(35, 35, 1030, 670)) 

# Constants
ONE_PIXEL_LEN = 0.112  # Length in reality corresponding to one pixel in BPU_Tool
DROPLET_HEIGHT = 0.2   # Height of the droplet
COEFFICIENT = math.pow(ONE_PIXEL_LEN, 2) * DROPLET_HEIGHT  # Coefficient for volume calculation
ONE_UL_LEN = 19.96489265625  # Edge length of 1 ul droplet on UI (assuming a square droplet)

START_x1, START_y1 = 180, 260
START_x2, START_y2 = 390, 260
START_x3, START_y3 = 600, 260
START_x4, START_y4 = 810, 260

list_start_x = list(range(180, 1020, 105))
list_start_y = [75, 400]

In [ ]:
api_handle.send_command(f"vbox add {1072 - 894} {409.5} {1072 - 992} {634.5}")

-1

In [ ]:
mag_centers = af.get_mag_info(8, 4)

In [ ]:

y_offset = 35

def get_mags(begin_row, begin_col, end_row, end_col, arrange=0):
    # 先定义整个32个点的矩阵
    global mag_centers
    matrix = mag_centers
    
    # 定义行列数
    rows, cols = 8, 4
    
    # 提取指定范围内的子矩阵
    sub_matrix = []
    
    if arrange == 0:  # 列优先
        for c in range(begin_col, end_col + 1):
            for r in range(begin_row, end_row + 1):
                index = c * rows + r
                sub_matrix.append(matrix[index])
    else:  # 行优先
        for r in range(begin_row, end_row + 1):
            for c in range(begin_col, end_col + 1):
                index = c * rows + r
                sub_matrix.append(matrix[index])
    
    return sub_matrix

def move_to_x(drp_id, x, wait = False):
    _, y1, _, y2 = api_handle.send_command(f"getbox {drp_id}")
    y = (y1 + y2) / 2
    tag = api_handle.send_command(f"move_p {drp_id} {x} {y} {MoveDirection.Horizontal.value}")
    if wait:
        api_handle.wait_tag(tag)
    return tag
    
def move_to_y(drp_id, y, wait = False):
    x1, _, x2, _ = api_handle.send_command(f"getbox {drp_id}")
    x = (x1 + x2) / 2
    tag = api_handle.send_command(f"move_p {drp_id} {x} {y} {MoveDirection.Vertical.value}")
    if wait:
        api_handle.wait_tag(tag)
    return tag

def get_4drps_list_vertical(begin, end):
    # 生成从 begin 到 end 的倒序列表
    num_list = list(range(end, begin - 1, -1))
    
    # 将数字分为两部分，分别为每列的数据
    col1 = num_list[::2]  # 选择奇数索引
    col2 = num_list[1::2]  # 选择偶数索引
    
    # 创建结果列表
    result = []
    
    # 每次处理4个元素
    for i in range(0, len(col1), 4):
        # 添加列1的4个元素
        result.append(col1[i:i+4])
        # 添加列2的4个元素
        result.append(col2[i:i+4])
    
    return result

def group_list(input_list):
    odd_index_elements = input_list[0::2]  # 选取奇数项索引的元素
    even_index_elements = input_list[1::2]  # 选取偶数项索引的元素
    return odd_index_elements, even_index_elements 

def split_into_groups(input_list, num_groups):
    # 计算每组的大小
    group_size = len(input_list) // num_groups
    # 创建组列表
    groups = [input_list[i*group_size:(i+1)*group_size] for i in range(num_groups)]
    # 将剩余的元素加入到最后一组（如果有多余的元素）
    remaining_elements = len(input_list) % num_groups
    if remaining_elements:
        groups[-1].extend(input_list[-remaining_elements:])
    
    return groups

def get_droplet_central_point(uid):
    uid = int(uid)  # 提前转换uid为整数
    x1, y1, x2, y2 = api_handle.send_command(f"getbox {uid}")
    # 计算中心点坐标并返回
    return (x1 + x2) / 2, (y1 + y2) / 2

# Convert a list of numbers to a space-separated string
def list_to_string(numbers):
    return ' '.join(map(str, numbers))

def get_area_central_point(area): 
    if (4 != len(area)) :
        return -1, -1
    return (area[0] + area[2]) / 2, (area[1] + area[3]) / 2

def process_list(input_list):
    # 使用列表解构和reversed，避免中间变量和原地修改
    g1, g2 = group_list(input_list)
    return list(reversed(g1)) + list(reversed(g2))

def move_drps_to_tmp_pos(drps_list: list, dest_mags: list, dir: str):
    tmp_tags = []
    dir_stringlist = ['up', 'right', 'left', 'right']
    sign = -1 if (dir == dir_stringlist[0] or dir == dir_stringlist[3]) else 1
    for drp_id, pos in zip(drps_list, dest_mags):
        # 将液滴摆放在目标磁控点的上下方向
        if (dir in dir_stringlist[:2]):
            tag = api_handle.send_command(f"move_p {drp_id} {pos[0]} {pos[1] + sign * y_offset} {MoveDirection.Vertical.value}")
            tmp_tags.append(tag)
        # 左右的逻辑还没想好
        else:
            pass
    af.wait_tag_all(tmp_tags)

def move_drps_to_points_v(drps: list, positions: list):
    tags = []
    for id, pos in zip(drps, positions):
        tag = api_handle.send_command(f"move_p {id} {list_to_string(pos)} {MoveDirection.Vertical.value}")
        tags.append(tag)
    af.wait_tag_all(tags)
    
def move_drps_to_points_h(drps: list, positions: list):
    tags = []
    for id, pos in zip(drps, positions):
        tag = api_handle.send_command(f"move_p {id} {list_to_string(pos)} {MoveDirection.Horizontal.value}")
        tags.append(tag)
    af.wait_tag_all(tags)

In [ ]:
def row_to_column_sort(data, rows = 6, cols = 4):
    """
    将一个按行排序的列表转换为按列排序
    
    参数:
    data -- 输入列表，按行排序的数据
    
    返回:
    列表，按列排序后的数据
    """
    # 确保输入是列表
    if not isinstance(data, list):
        raise TypeError("输入必须是列表")
    
    # 将输入列表转换为NumPy数组，假设是6×4的形状
    arr = np.array(data).reshape(rows, cols)
    
    # 将数组按列展开，即先取所有行的第0列，然后是第1列，依此类推
    column_ordered = arr.T.flatten()
    
    # 转换回Python列表
    result = column_ordered.tolist()
    
    return result


In [ ]:
def get_hole_pos(dir: str, idx: int) -> list:
    """
    获取芯片孔位坐标
    
    参数:
        dir (str): 方向 - 'l'左, 'r'右, 'u'上, 'd'下
        idx (int): 孔位索引，从1开始
        
    返回:
        list: [x坐标, y坐标]
    """
    # 定义各个方向的孔位坐标
    holes_config = {
        'r': {
            'x': 988,
            'y': [115, 225, 330, 435, 545, 620]
        },
        'l': {
            'x': 75,
            'y': [85, 175, 280, 385, 490, 595]
        },
        'd': {
            'x': [100, 190, 295, 400, 510, 615, 725, 830, 940],
            'y': 640
        },
        'u': {
            'x': [135, 240, 345, 455, 560, 670, 775, 885, 965],
            'y': 70
        }
    }
    
    # 检查参数有效性
    if dir not in holes_config:
        raise ValueError("方向参数必须是 'l', 'r', 'u' 或 'd'")
    
    holes = holes_config[dir]
    
    # 获取该方向的最大索引值
    max_idx = len(holes['y']) if isinstance(holes['x'], int) else len(holes['x'])
    
    if idx < 1 or idx > max_idx:
        raise ValueError(f"索引值必须在1到{max_idx}之间")
    
    # 将索引转换为0基数
    idx = idx - 1
    
    # 返回坐标
    if isinstance(holes['x'], int):
        # 左右方向：x坐标固定，y坐标是列表
        return [holes['x'], holes['y'][idx]]
    else:
        # 上下方向：x坐标是列表，y坐标固定
        return [holes['x'][idx], holes['y']]

def get_hole_pos_by_pipe_id(pipe_id: int):
    if (0 != pipe_id % 2):
        return [-1, -1]
    # 左侧孔位坐标
    if (pipe_id <= 10):
        return get_hole_pos('r', (10 - pipe_id) // 2 + 1)
    elif (pipe_id >= 12 and pipe_id <= 28):
        return get_hole_pos('u', (28 - pipe_id) // 2 + 1)
    elif (pipe_id >= 30 and pipe_id <= 40):
        return get_hole_pos('l', (pipe_id - 30) // 2 + 1)
    elif (pipe_id >= 42 and pipe_id <= 58):
        return get_hole_pos('d', (pipe_id - 42) // 2 + 1)

def wait_tag(tag: int):
    api_handle.wait_tag(tag)

In [ ]:
def single_mix(drp_id, mix_time):
    x1, y1, x2, y2 = api_handle.send_command(f"getbox {drp_id}")
    api_handle.send_command(f"set_virtual {drp_id} {0}")
    move_length = 0.7
    list_direction = [(move_length*(-1),0), (-move_length*2**0.5/2,move_length*2**0.5/2), (0,move_length), (move_length*2**0.5/2,move_length*2**0.5/2), (move_length,0), (move_length*2**0.5/2,-move_length*2**0.5/2), (0,move_length*(-1)), (-move_length*2**0.5/2,-move_length*2**0.5/2), (move_length*(-1),0), (-move_length*2**0.5/2,-move_length*2**0.5/2), (0,move_length*(-1)), (move_length*2**0.5/2,-move_length*2**0.5/2), (move_length,0), (move_length*2**0.5/2,move_length*2**0.5/2), (0,move_length), (-move_length*2**0.5/2,move_length*2**0.5/2)]
    running_time = 0
    time_interval = 0.4
    while running_time <= mix_time:
        for direction in list_direction:
            sx_left = x1+(x2-x1)*direction[0]
            sx_right = sx_left + x2 - x1
            sy_right = y1+(y2-y1)*direction[1]
            sy_up = sy_right + y2 - y1
            vbox_id = api_handle.send_command(f"vbox add {sx_left} {sy_right} {sx_right} {sy_up}")
            time.sleep(time_interval)
            api_handle.send_command(f"vbox remove {vbox_id}")
            running_time += time_interval
    vbox_id = api_handle.send_command(f"vbox add {x1} {y1} {x2} {y2}")
    drp_id_new = api_handle.send_command(f"vbox 2drp {vbox_id}")
    return drp_id_new

def batch_mix(list_drp_id, mix_time):
    list_feature = []
    with ThreadPoolExecutor(max_workers=32) as executor:
        for drp_id in list_drp_id:
            feature = executor.submit(single_mix, drp_id, mix_time)
            list_feature.append(feature)
    list_drp_id_new = []
    for feature in list_feature:  
        list_drp_id_new.append(feature.result())
    return list_drp_id_new
mix_func = batch_mix

In [ ]:
def move_drps_to_points(drps: list, positions: list, path_temp_point = False, temp_point_dir: str = 'up'):
    if path_temp_point:
        move_drps_to_tmp_pos(drps, positions, temp_point_dir)
    tags = []
    for id, pos in zip(drps, positions):
        tag = api_handle.send_command(f"move_p {id} {list_to_string(pos)} {MoveDirection.Vertical.value}")
        tags.append(tag)
    af.wait_tag_all(tags)

def move_vbox_p(vbox_id, target_x, target_y, dir: MoveDirection, speed = 2):
    curr_x, curr_y = get_area_central_point(api_handle.send_command(f"vbox getbox {vbox_id}"))
    if (MoveDirection.Horizontal.value == dir):
        api_handle.wait_tag( \
        api_handle.send_command(f"vbox move {vbox_id} {target_x} {curr_y} {speed}")
        )
        api_handle.wait_tag( \
        api_handle.send_command(f"vbox move {vbox_id} {target_x} {target_y} {speed}")
        )
    else:
        api_handle.wait_tag( \
        api_handle.send_command(f"vbox move {vbox_id} {curr_x} {target_y}")
        )
        api_handle.wait_tag( \
        api_handle.send_command(f"vbox move {vbox_id} {target_x} {target_y}")
        )

def move_shift_vbox(vbox_id, x_shift, y_shift, speed = 4):
    curr_x, curr_y = get_area_central_point(api_handle.send_command(f"vbox getbox {vbox_id}"))
    return api_handle.send_command(f"vbox move {vbox_id} {curr_x + x_shift} {curr_y + y_shift} {speed}")

In [ ]:
split_volume = 1.67
orig_volume = split_volume * 9
orig_length = ONE_UL_LEN * orig_volume**0.5
split_length = ONE_UL_LEN * split_volume**0.5

matrix = af.get_mag_info(8, 4)
speed_drp = 0.25
speed_vbox = 2
matrix.extend([[853, 601], [853, 530], [853, 459], [853, 388], [853, 317], [853, 246], [853, 175], [853, 104]])

函数定义

In [ ]:
def vbox_move_shift(vbox_id, x_distance, y_distance, speed):
    x1, y1, x2, y2 = api_handle.send_command("vbox getbox {}".format(vbox_id))
    x_destination = (x1 + x2)/2 + x_distance
    y_destination = (y1 + y2)/2 + y_distance
    tag = api_handle.send_command(f"vbox move {vbox_id} {x_destination} {y_destination} {speed}")
    return tag

def extract_droplet_v2(vbox_id, direction, orig_volume, split_volume, split_number, list_split_position, extract_time = 3):
    x1, y1, x2, y2 = api_handle.send_command("vbox getbox {}".format(vbox_id))
    api_handle.send_command(f"vbox remove {vbox_id}")

    list_vbox_id_saved = []
    step = 0
    while step < split_number:
        area = orig_volume - step * split_volume
        length_new = ONE_UL_LEN*(area-split_volume)**0.5
        split_length = ONE_UL_LEN * split_volume**0.5
        speed = 2
        sx_left = (x1+x2)*0.5 - length_new*0.5
        sx_right = (x1+x2)*0.5 + length_new*0.5
        sy_down = (y1+y2)*0.5 - length_new*0.5
        sy_up = (y1+y2)*0.5 + length_new*0.5
        # bottom_vbox
        if direction == 'down':
            bottom_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left,sy_down,sx_right,sy_down+length_new*0.1))
        elif direction == 'up':
            bottom_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left,sy_up-length_new*0.1,sx_right,sy_up))
        elif direction == 'left':
            bottom_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_right-length_new*0.1,sy_down,sx_right,sy_up))
        elif direction == 'right':
            bottom_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left,sy_down,sx_left+length_new*0.1,sy_up))            
        # neck_vbox
        if direction == 'down':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/4,sy_up-split_length*0.5,(x1+x2)*0.5+split_length/4,sy_up+split_length*1.5))
        elif direction == 'up':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/4,sy_down-split_length*1.5,(x1+x2)*0.5+split_length/4,sy_down+split_length*0.5))
        elif direction == 'left':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left-split_length*1.5,(y1+y2)/2-split_length/4,sx_left+split_length*0.5,(y1+y2)/2+split_length/4))
        elif direction == 'right':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_right-split_length*0.5,(y1+y2)/2-split_length/4,sx_right+split_length*1.5,(y1+y2)/2+split_length/4))
        time.sleep(1)
        
        # vbox_saved
        if direction == 'down':
            vbox_id_saved = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/2,sy_up+split_length*1,(x1+x2)*0.5+split_length/2,sy_up+split_length*2))
        elif direction == 'up':
            vbox_id_saved = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/2,sy_down-split_length*2,(x1+x2)*0.5+split_length/2,sy_down-split_length))
        elif direction == 'left':
            vbox_id_saved = api_handle.send_command("vbox add {} {} {} {}".format(sx_left-split_length*2,(y1+y2)*0.5-split_length/2,sx_left-split_length,(y1+y2)*0.5+split_length/2))
        elif direction == 'right':
            vbox_id_saved = api_handle.send_command("vbox add {} {} {} {}".format(sx_right+split_length,(y1+y2)*0.5-split_length/2,sx_right+split_length*2,(y1+y2)*0.5+split_length/2))
        list_vbox_id_saved.append(vbox_id_saved)
        time.sleep(extract_time)
        api_handle.send_command(f"vbox remove {neck_vbox_id}")
        api_handle.send_command(f"vbox remove {bottom_vbox_id}")
        
        # second_neck_vbox
        if direction == 'down':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/16,sy_up-split_length*0.5,(x1+x2)*0.5+split_length/16,sy_up+split_length*1))
        elif direction == 'up':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/16,sy_down-split_length*1,(x1+x2)*0.5+split_length/16,sy_down+split_length*0.5))
        elif direction == 'left':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left-split_length*1,(y1+y2)/2-split_length/16,sx_right-split_length*0.5,(y1+y2)/2+split_length/16))
        elif direction == 'right':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_right-split_length*0.5,(y1+y2)/2-split_length/16,sx_right+split_length*1,(y1+y2)/2+split_length/16))
            
        bottom_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left,sy_down,sx_right,sy_up))
        time.sleep(2)
        api_handle.send_command(f"vbox remove {neck_vbox_id}")
        
        # withdraw_vbox
        if direction == 'down':
            withdraw_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length*0.5,sy_up-split_length*0.5,(x1+x2)*0.5+split_length*0.5,sy_up+split_length*0.5))
        elif direction == 'up':
            withdraw_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length*0.5,sy_down-split_length*0.5,(x1+x2)*0.5+split_length*0.5,sy_down+split_length*0.5))
        elif direction == 'left':
            withdraw_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left-split_length*0.5,(y1+y2)/2-split_length*0.5,sx_left+split_length*0.5,(y1+y2)/2+split_length*0.5))
        elif direction == 'right':
            withdraw_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_right-split_length/2,(y1+y2)/2-split_length*0.5,sx_right+split_length/2,(y1+y2)/2+split_length*0.5))
        time.sleep(0.5)
        api_handle.send_command(f"vbox remove {withdraw_vbox_id}")
        
        # move
        tag = None
        if direction == 'up':
            tag = api_handle.send_command(f"vbox move {vbox_id_saved} {list_split_position[step][0]} {list_split_position[step][1]} {speed}")
        elif direction == 'down':
            tag = api_handle.send_command(f"vbox move {vbox_id_saved} {list_split_position[split_number-step-1][0]} {list_split_position[split_number-step-1][1]} {speed}")
        elif direction == 'left':
            tag = api_handle.send_command(f"vbox move {vbox_id_saved} {list_split_position[step][0]} {list_split_position[step][1]} {speed}")
        elif direction == 'right':
            tag = api_handle.send_command(f"vbox move {vbox_id_saved} {list_split_position[split_number-step-1][0]} {list_split_position[split_number-step-1][1]} {speed}")
            
        time.sleep(5)
        api_handle.send_command(f"vbox remove {bottom_vbox_id}")
        if step == split_number-1:
            rest_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left,sy_down,sx_right,sy_up))
            api_handle.wait_tag(tag)
        step += 1
        
    if direction == 'up' or direction == 'left':
        return list_vbox_id_saved, rest_vbox_id
    else:
        return list_vbox_id_saved[::-1], rest_vbox_id


In [ ]:
def even_split(vbox_id, move_speed):
    x1, y1, x2, y2 = api_handle.send_command(f"vbox getbox {vbox_id}")
    api_handle.send_command(f"vbox remove {vbox_id}")
    length_vbox_end = (x2-x1)/2**0.5
    bar_height = 2
    vbox1 = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)/2-length_vbox_end/2,(y1+y2)/2-length_vbox_end/2,(x1+x2)/2+length_vbox_end/2,(y1+y2)/2+length_vbox_end/2))
    vbox2 = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)/2-length_vbox_end/2,(y1+y2)/2-length_vbox_end/2,(x1+x2)/2+length_vbox_end/2,(y1+y2)/2+length_vbox_end/2))
    vbox3 = api_handle.send_command("vbox add {} {} {} {}".format(x1,(y1+y2)/2-bar_height/2,x2,(y1+y2)/2+bar_height/2))
    time.sleep(1)
    tags = []
    tag = move_shift_vbox(vbox1, -(x2-x1)*0.75, 0, move_speed)
    tags.append(tag)
    tag = move_shift_vbox(vbox2, (x2-x1)*0.75, 0, move_speed)
    tags.append
    af.wait_tag_all(tags)
    time.sleep(1)
    api_handle.send_command(f"vbox remove {vbox3}") 
    return vbox1, vbox2

def batch_even_split(list_vbox_id, move_speed):
    list_feature = []
    with ThreadPoolExecutor(max_workers=128) as executor:
        for vbox_id in list_vbox_id:
            feature = executor.submit(even_split, vbox_id, move_speed)
            list_feature.append(feature)
    list_vbox_id_new = []
    for feature in list_feature:  
        list_vbox_id_new.append(feature.result())
    return list_vbox_id_new

def even_split_2(drp_id, move_speed):
    x1, y1, x2, y2 = api_handle.send_command(f"getbox {drp_id}")
    api_handle.send_command(f"set_virtual {drp_id} 1")
    length_vbox_end = (x2-x1)/2**0.5
    bar_height = 2
    vbox1 = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)/2-length_vbox_end/2,(y1+y2)/2-length_vbox_end/2,(x1+x2)/2+length_vbox_end/2,(y1+y2)/2+length_vbox_end/2))
    vbox2 = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)/2-length_vbox_end/2,(y1+y2)/2-length_vbox_end/2,(x1+x2)/2+length_vbox_end/2,(y1+y2)/2+length_vbox_end/2))
    vbox3 = api_handle.send_command("vbox add {} {} {} {}".format(x1,(y1+y2)/2-bar_height/2,x2,(y1+y2)/2+bar_height/2))
    time.sleep(1)
    tags = []
    tag = move_shift_vbox(vbox1, -(x2-x1)*0.75, 0, move_speed)
    tags.append(tag)
    tag = move_shift_vbox(vbox2, (x2-x1)*0.75, 0, move_speed)
    tags.append
    af.wait_tag_all(tags)
    time.sleep(1)
    api_handle.send_command(f"vbox remove {vbox3}")
    id1 = api_handle.send_command(f"vbox 2drp {vbox1}")
    id2 = api_handle.send_command(f"vbox 2drp {vbox2}")
    return id1, id2

def batch_even_split_2(list_drp_id, move_speed):
    list_feature = []
    with ThreadPoolExecutor(max_workers=128) as executor:
        for drp_id in list_drp_id:
            feature = executor.submit(even_split_2, drp_id, move_speed)
            list_feature.append(feature)
    list_drp_id_new = []
    for feature in list_feature:
        list_drp_id_new.append(feature.result())
    return list_drp_id_new

In [ ]:
def garbage_drps(drps: list, move_shift_x = 300, mag_up: bool = True):
    def garbage_func():
        if mag_up:
            api_handle.send_command("mag up")
            time.sleep(6)
        # af.move_shift_all(drps, -35, 0, True)
        # time.sleep(1)
        # af.move_shift_all(drps, 0, 35, True)
        # time.sleep(1)
        # af.move_shift_all(drps, 70, -70, True)
        # time.sleep(2)
        drps2 = mix_func(drps, 10)
        af.move_shift_all(drps2, 35, -35, True)
        time.sleep(2)
        
        if move_shift_x > 0:
            af.move_shift_all(drps2, move_shift_x, 0, True)
        # api_handle.send_command("mag down")
        
        y1 = get_droplet_central_point(drps2[0])[1]
        garbage_pos_x, garbage_pos_y = get_hole_pos_by_pipe_id(12)[0], 62
        def move_to_garbage_pos(id):
            tag = api_handle.send_command(f"move_p {id} {garbage_pos_x} {y1} {MoveDirection.Vertical.value}")
            api_handle.wait_tag(tag)
            tag = api_handle.send_command(f"move_p {id} {garbage_pos_x} {garbage_pos_y} {MoveDirection.Vertical.value}")
            api_handle.wait_tag(tag)
            api_handle.send_command(f"set_virtual {id} 0")
            
        x_st = get_droplet_central_point(drps2[-1])[0]
        for i in range(len(drps2) // 6):
            st = -(i+1)*6
            ed = -i*6 if i != 0 else None
            group = drps2[st:ed]
            for j, id in enumerate(group):
                t = threading.Thread(target = move_to_garbage_pos, args = (id, ))
                t.start()
            while(get_droplet_central_point(group[-1])[0] < x_st + 30):
                time.sleep(1)
            if (i != len(drps2) // 6 - 1):
                af.move_shift_all(drps2[:len(drps2) - (i + 1) * 6], 70, 0, True)
                     
    t = threading.Thread(target=garbage_func)
    t.start()
    return t

In [ ]:
split_volume = 1
drp_num = 6

In [ ]:
speed = 1.2